In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from scipy.interpolate import make_interp_spline

In [ ]:
# path containing csv files exported from scherrer script 
# the first column is thickness values in nm
# the following colums are as follows : m, c, A, mu, sigma, sigma_error, FWHM, FWHM_error, D, D_error


# plots all the csvs in a given folder
folder_path = '/home/goughl/Documents/ea_july/'
filenames = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

# labels based on filenames (this part needs improving)
labels = [os.path.splitext(os.path.basename(f))[0] for f in filenames]
dataframes = [pd.read_csv(f) for f in filenames]

# optional : separate labels and dataframes into EA and non-EA groups
# change as required
ea_data = [(df, label) for df, label in zip(dataframes, labels) if 'EA' in label]
non_ea_data = [(df, label) for df, label in zip(dataframes, labels) if 'EA' not in label]

# assign colour gradients
ea_colors = cm.Reds(np.linspace(0.4, 0.9, len(ea_data)))
non_ea_colors = cm.Blues(np.linspace(0.4, 0.9, len(non_ea_data)))



In [ ]:
##### plotting D with D_error

plt.figure(figsize=(8, 6))
for (df, label), color in zip(non_ea_data, non_ea_colors):
    plt.errorbar(df.iloc[:, 0], df['D'], yerr=df['D_error'], marker='^', capsize=4, label=label, color=color, linestyle='--')
for (df, label), color in zip(ea_data, ea_colors):
    plt.errorbar(df.iloc[:, 0], df['D'], yerr=df['D_error'], marker='^', capsize=4, label=label, color=color, linestyle='--')
plt.xlabel('Film Thickness (nm)')
plt.ylabel('Crystallite Coherence Length (nm)')
#plt.title('Crystallite Size vs Thickness')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
#### plotting mu with sigma_error
plt.figure(figsize=(8, 6))
for (df, label), color in zip(non_ea_data, non_ea_colors):
    plt.errorbar(df.iloc[:, 0], df['mu'], yerr=df['sigma_error'], marker='^', capsize=4, label=label, color=color, linestyle='--')
for (df, label), color in zip(ea_data, ea_colors):
    plt.errorbar(df.iloc[:, 0], df['mu'], yerr=df['sigma_error'], marker='^', capsize=4, label=label, color=color,linestyle='--')
plt.xlabel('Film Thickness (nm)')
plt.ylabel('Peak Position (1/Å)')
#plt.title('Peak Position vs Thickness')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
##### plotting A

plt.figure(figsize=(8, 6))
for (df, label), color in zip(non_ea_data, non_ea_colors):
    plt.plot(df.iloc[:, 0], df['A'], marker='^', linestyle='--', label=label, color=color)
for (df, label), color in zip(ea_data, ea_colors):
    plt.plot(df.iloc[:, 0], df['A'], marker='^', linestyle='--', label=label, color=color)
plt.xlabel('Film Thickness (nm)')
plt.ylabel('Peak Amplitude (a.u)')
#plt.title('Amplitude vs Thickness')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
### normalise amplitude to a given thickness ####

ref_thickness = 10 # <-- Set this to your desired reference thickness in nm

plt.figure(figsize=(8, 6))


for (df, label), color in zip(non_ea_data, non_ea_colors):
    # find closest thickness index
    closest_idx = (df.iloc[:, 0] - ref_thickness).abs().idxmin()
    ref_value = df.loc[closest_idx, 'A']
    
    # avoid division by zero
    if ref_value != 0:
        norm_A = df['A'] / ref_value
        plt.plot(df.iloc[:, 0], norm_A, marker='^', linestyle='--', label=label, color=color)
    else:
        print(f"Warning: Reference amplitude is zero in {label}. Skipping normalization.")

for (df, label), color in zip(ea_data, ea_colors):
    closest_idx = (df.iloc[:, 0] - ref_thickness).abs().idxmin()
    ref_value = df.loc[closest_idx, 'A']
    
    if ref_value != 0:
        norm_A = df['A'] / ref_value
        plt.plot(df.iloc[:, 0], norm_A, marker='^', linestyle='--', label=label, color=color)
    else:
        print(f"Warning: Reference amplitude is zero in {label}. Skipping normalization.")

plt.xlabel('Film Thickness (nm)')
plt.ylabel(f'Normalised Peak Amplitude (a.u.)')
#plt.ylabel(f'Normalised Peak Amplitude (relative to {ref_thickness} nm)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:


# # === Load CSVs ===
# folder_path = '/home/goughl/Documents/ea_july/thick'
# filenames = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
# labels = [os.path.splitext(os.path.basename(f))[0] for f in filenames]
# dataframes = [pd.read_csv(f) for f in filenames]

# # === Categorize datasets ===
# ea_data = [(df, label) for df, label in zip(dataframes, labels) if 'IP' in label]
# non_ea_data = [(df, label) for df, label in zip(dataframes, labels) if 'IP' not in label]

# # Split EA group into lamellar and non-lamellar
# ea_dark = [(df, label) for df, label in ea_data if 'lamellar' in label.lower()]
# ea_light = [(df, label) for df, label in ea_data if 'lamellar' not in label.lower()]

# # Assign color gradients
# ea_dark_colors = cm.Reds(np.linspace(0.7, 0.9, len(ea_dark)))    # Darker red for lamellar
# ea_light_colors = cm.Reds(np.linspace(0.4, 0.6, len(ea_light)))  # Lighter red for others
# non_ea_colors = cm.Blues(np.linspace(0.4, 0.9, len(non_ea_data))) # Blues for non-IP

# # === Plot helper with smoothing and error bars ===
# def plot_smoothed(ax, df, xcol, ycol, yerrcol, label, color, marker):
#     x = df[xcol]
#     y = df[ycol]
#     yerr = df[yerrcol] if yerrcol else None

#     if len(x) >= 4:
#         x_smooth = np.linspace(x.min(), x.max(), 20)
#         spline = make_interp_spline(x, y, k=2)
#         y_smooth = spline(x_smooth)
#         ax.plot(x_smooth, y_smooth, color=color, label=label, linestyle='--')

#     ax.errorbar(x, y, yerr=yerr, fmt=marker, capsize=4, color=color)

# # === Plot D ===
# fig, ax = plt.subplots(figsize=(8, 6))
# for (df, label), color in zip(non_ea_data, non_ea_colors):
#     plot_smoothed(ax, df, xcol=df.columns[0], ycol='D', yerrcol='D_error', label=label, color=color, marker='^')
# for (df, label), color in zip(ea_light, ea_light_colors):
#     plot_smoothed(ax, df, xcol=df.columns[0], ycol='D', yerrcol='D_error', label=label, color=color, marker='^')
# for (df, label), color in zip(ea_dark, ea_dark_colors):
#     plot_smoothed(ax, df, xcol=df.columns[0], ycol='D', yerrcol='D_error', label=label, color=color, marker='^')
# ax.set_xlabel('Film Thickness (nm)')
# ax.set_ylabel('Crystallite Coherence Length D (nm)')
# ax.legend()
# ax.grid(True)
# plt.tight_layout()
# plt.show()

# # # === Plot μ with σ_error ===
# # fig, ax = plt.subplots(figsize=(8, 6))
# # for (df, label), color in zip(non_ea_data, non_ea_colors):
# #     plot_smoothed(ax, df, xcol=df.columns[0], ycol='mu', yerrcol='sigma_error', label=label, color=color, marker='s')
# # for (df, label), color in zip(ea_light, ea_light_colors):
# #     plot_smoothed(ax, df, xcol=df.columns[0], ycol='mu', yerrcol='sigma_error', label=label, color=color, marker='s')
# # for (df, label), color in zip(ea_dark, ea_dark_colors):
# #     plot_smoothed(ax, df, xcol=df.columns[0], ycol='mu', yerrcol='sigma_error', label=label, color=color, marker='s')
# # ax.set_xlabel('Film Thickness (nm)')
# # ax.set_ylabel('Peak Position μ (1/nm)')
# # ax.legend()
# # ax.grid(True)
# # plt.tight_layout()
# # plt.show()

# # === Plot A (no error bars) ===
# fig, ax = plt.subplots(figsize=(8, 6))
# for (df, label), color in zip(non_ea_data, non_ea_colors):
#     plot_smoothed(ax, df, xcol=df.columns[0], ycol='A', yerrcol=None, label=label, color=color, marker='^')
# for (df, label), color in zip(ea_light, ea_light_colors):
#     plot_smoothed(ax, df, xcol=df.columns[0], ycol='A', yerrcol=None, label=label, color=color, marker='^')
# for (df, label), color in zip(ea_dark, ea_dark_colors):
#     plot_smoothed(ax, df, xcol=df.columns[0], ycol='A', yerrcol=None, label=label, color=color, marker='^')
# ax.set_xlabel('Film Thickness (nm)')
# ax.set_ylabel('Amplitude A')
# ax.legend()
# ax.grid(True)
# plt.tight_layout()
# plt.show()
